In [76]:
# !pip install -r requirements.txt -q

In [77]:
from ultralytics import YOLO
import cv2
import random
import numpy as np
from dotenv import load_dotenv
import os
from inference_sdk import InferenceHTTPClient

In [78]:
from segmentator import Segmentator, find_tangent_lines, HeadSegmentator, px_in_cm, get_head_edge_points, get_head_highest_points
from detector import Detector

In [79]:
segm = Segmentator(model_path="weights/segmentation.pt")
det = Detector(model_id="hip-joint-detection/2")
head_segm = HeadSegmentator(model_id="femur-head-segmentation/2")

In [80]:
image1 = 'test_data/img1.jpg'
image2 = 'test_data/img2.jpg'
image3 = 'test_data/img3.jpg'
detected = 'test_data/detected.jpg'
segmented = 'test_data/segmented.jpg'

Detect all results

In [81]:
image = cv2.imread(image1)
det_results = det.detect(image1)
head_segm_results = head_segm.segment(image1)
segm_results = segm.segment(image1)
_, left_line, right_line, left_hole_top_left, right_hole_top_right, new_points = find_tangent_lines(image1, segm_results, det_results)
cm = px_in_cm(det_results)
head_edge_points = get_head_edge_points(head_segm_results)
left_edge_point = min(head_edge_points[0], key=lambda x: x['x'])
print("left_edge_point", left_edge_point)
right_edge_point =  max(head_edge_points[1], key=lambda x: x['x'])
print("right_edge_point", right_edge_point)
head_highest_points = get_head_highest_points(head_segm_results, cm) 
left_highest_point, right_highest_point = head_highest_points[0], head_highest_points[1]
print("left_highest_point", left_highest_point)
print("right_highest_point", right_highest_point)
left_right_highest_point = new_points[0]
print("left_right_highest_point", left_right_highest_point)
right_left_highest_point = new_points[1]
print("right_left_highest_point", right_left_highest_point)


0: 640x640 1 big-hole, 2 bone-hollows, 652.6ms
Speed: 4.5ms preprocess, 652.6ms inference, 11.6ms postprocess per image at shape (1, 3, 640, 640)
((209, 238), (19, -743)) left
left_edge_point {'x': 101.0, 'y': 200.0}
right_edge_point {'x': 534.0, 'y': 201.0}
{'x': 132.0, 'y': 173.0}
{'x': 475.0, 'y': 169.0}
left_highest_point (132.0, 163.685)
right_highest_point (475.0, 159.685)
left_right_highest_point (19, -743)
right_left_highest_point (642, -734)


In [82]:
left_left_top = (int(left_edge_point['x']), int(left_highest_point[1]))
left_left_bottom = (int(left_edge_point['x']), int(left_hole_top_left[1]))
left_right_top = (int(left_right_highest_point[0]),int(left_highest_point[1]))

dist_left = left_left_top[0] - left_right_top[0]
left_right_top = (int(left_right_highest_point[0] + 2*dist_left + 10), int(left_highest_point[1]))

print("left_right_top", left_right_top)
left_right_bottom = (int(left_hole_top_left[0]), int(left_hole_top_left[1]))

right_right_top = (int(right_edge_point['x']), int(right_highest_point[1]))
right_right_bottom = (int(right_edge_point['x']), int(right_hole_top_right[1]))
right_left_top = (int(right_left_highest_point[0]), int(right_highest_point[1]))
dist_right = right_right_top[0] - right_left_highest_point[0] 
right_left_top = (int(right_left_highest_point[0] + 2*dist_right + 10), int(right_highest_point[1]))

print("right_left_top", right_left_top)  
# right_left_top = (int(right_left_highest_point[0] - 2*dist_right - 10), int(right_highest_point[1]))
right_left_bottom = (int(right_hole_top_right[0]), int(right_hole_top_right[1]))


left_right_top (193, 163)
right_left_top (436, 159)


In [83]:
cv2.circle(image, left_left_top, 5, (0, 0, 255), -1)
cv2.circle(image, left_left_bottom, 5, (0, 0, 255), -1)
cv2.circle(image, left_right_top, 5, (0, 255, 0), -1)
cv2.circle(image, left_right_bottom, 5, (0, 0, 255), -1)

cv2.circle(image, right_right_top, 5, (0, 0, 255), -1)
cv2.circle(image, right_right_bottom, 5, (0, 0, 255), -1)
cv2.circle(image, right_left_top, 5, (0, 255, 0), -1)
cv2.circle(image, right_left_bottom, 5, (0, 0, 255), -1)


array([[[ 16,  81,  12],
        [ 73, 134,  70],
        [  0,  38,   0],
        ...,
        [ 58, 114,  61],
        [ 64, 126,  66],
        [140, 208, 143]],

       [[  0,  45,   0],
        [  0,  51,   0],
        [  0,  20,   0],
        ...,
        [  9,  59,  11],
        [ 45, 102,  47],
        [129, 191, 131]],

       [[  0,  27,   0],
        [  0,  20,   0],
        [  0,  17,   0],
        ...,
        [  0,  21,   0],
        [  0,  33,   0],
        [147, 199, 146]],

       ...,

       [[  0,  23,   0],
        [  0,  42,   0],
        [  0,  16,   0],
        ...,
        [  0,   9,   0],
        [ 55,  92,  54],
        [143, 185, 137]],

       [[ 23,  81,  22],
        [ 39,  94,  37],
        [  0,  27,   0],
        ...,
        [  0,  21,   0],
        [ 75, 118,  73],
        [152, 203, 146]],

       [[  0,  38,   0],
        [ 17,  77,  17],
        [  0,  23,   0],
        ...,
        [  0,  23,   0],
        [  0,  43,   0],
        [143, 197, 138]]

In [84]:

cv2.line(image, left_line[0], left_line[1], (255, 0, 0), 2)
cv2.line(image, right_line[0], right_line[1], (0, 255, 0), 2)

array([[[ 16,  81,  12],
        [ 73, 134,  70],
        [  0,  38,   0],
        ...,
        [ 58, 114,  61],
        [ 64, 126,  66],
        [140, 208, 143]],

       [[  0,  45,   0],
        [  0,  51,   0],
        [  0,  20,   0],
        ...,
        [  9,  59,  11],
        [ 45, 102,  47],
        [129, 191, 131]],

       [[  0,  27,   0],
        [  0,  20,   0],
        [  0,  17,   0],
        ...,
        [  0,  21,   0],
        [  0,  33,   0],
        [147, 199, 146]],

       ...,

       [[  0,  23,   0],
        [  0,  42,   0],
        [  0,  16,   0],
        ...,
        [  0,   9,   0],
        [ 55,  92,  54],
        [143, 185, 137]],

       [[ 23,  81,  22],
        [ 39,  94,  37],
        [  0,  27,   0],
        ...,
        [  0,  21,   0],
        [ 75, 118,  73],
        [152, 203, 146]],

       [[  0,  38,   0],
        [ 17,  77,  17],
        [  0,  23,   0],
        ...,
        [  0,  23,   0],
        [  0,  43,   0],
        [143, 197, 138]]

In [85]:
cv2.imwrite('test_data/tangent_lines.jpg', image)

True